# Processing, Clustering and Visualising in MDV 3k PBMCs

The part of this tutorial dedicated to analysing the data is based on the scanpy tutorial "Preprocessing and clustering 3k PBMCs (legacy workflow)" found here: https://scanpy.readthedocs.io/en/stable/tutorials/index.html . If you are interested in understanding the data analysis part of this tutorial more, please check the scanpy tutorial. The data consists of 3k PBMCs from a Healthy Donor and are available from 10x Genomics. On a unix system, you can uncomment and run the following to download and unpack the data. Alternatively you can download the data from http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz, create a new directory named "data" in the same level that this jupyter notebook is saved and run from, and unzip the data within the "data" directory.

In [2]:
import os
import requests
import tarfile
try:
    os.mkdir('data')
    os.mkdir('write')
except FileExistsError:
    pass
url = 'http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz'
r = requests.get(url)
with open('data/pbmc3k_filtered_gene_bc_matrices.tar.gz', 'wb') as f:
    f.write(r.content)
tar = tarfile.open('data/pbmc3k_filtered_gene_bc_matrices.tar.gz')
tar.extractall('data')
tar.close()
print('Downloaded and extracted data')
#!mkdir data
#!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
#!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
#!mkdir write

Downloaded and extracted data


#### Importing the required packages for data preprocessing

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc

#### Importing the required packages for MDV set up and visualisation

In [3]:
import os
import json
from mdvtools.mdvproject import MDVProject
from mdvtools.charts.row_chart import RowChart
from mdvtools.charts.dot_plot import DotPlot
from mdvtools.charts.histogram_plot import HistogramPlot
from mdvtools.charts.scatter_plot_3D import ScatterPlot3D
from mdvtools.charts.scatter_plot import ScatterPlot
from mdvtools.charts.table_plot import TablePlot

## Data analysis section

In [4]:
# scanpy parameters for feedback level setting
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header() # printing a header of introductory information about the environment/library used

/Users/mariak/Documents/MDVmk/MDV/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.10.1 anndata==0.9.2 umap==0.5.6 numpy==1.26.4 scipy==1.13.0 pandas==2.2.2 scikit-learn==1.4.2 statsmodels==0.14.2 igraph==0.11.4 pynndescent==0.5.12


In [5]:
#file_path = os.path.abspath('') # absolute path of the current python executing script file 
#dir = os.path.dirname(file_path) # absolute path of the directory containing the current script file
#os.chdir(dir) # this function changes the current working directory to the one specified in its argument

In [6]:
print(dir)

<built-in function dir>


In [7]:
# reading the 3k 10x data

adata = sc.read_10x_mtx(
    "data/filtered_gene_bc_matrices/hg19/",  # the directory with the `.mtx` file
    var_names="gene_symbols",  # use gene symbols for the variable names (variables-axis index)
    cache=True,)  # write a cache file for faster subsequent reading)

... reading from cache file cache/data-filtered_gene_bc_matrices-hg19-matrix.h5ad


#### Data preprocessing

In [8]:
# annotate the group of mitochondrial genes as "mt"
adata.var["mt"] = adata.var_names.str.startswith("MT-")

# running quality control metrics to filter the anndata object
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

# using filter_cells and filter_genes to filter our cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# slicing the anndata object to remove genes that have too many mitochondrial genes expressed or too many total counts
adata = adata[adata.obs.n_genes_by_counts < 2500, :] 
adata = adata[adata.obs.pct_counts_mt < 5, :].copy()

# total-count normalising the anndata object to 10,000 reads per cell, so that counts become comparable among cells
sc.pp.normalize_total(adata, target_sum=1e4)

# logarithmising the data
sc.pp.log1p(adata)

# identifying highly-variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# freezing the state of the anndata object to the normalised and logarithmised raw gene expression
adata.raw = adata

# filtering the highly-variable genes
adata = adata[:, adata.var.highly_variable]

# regressing out effects of total counts per cell and the percentage of mitochondrial genes expressed
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

# scaling the data to unit variance
sc.pp.scale(adata, max_value=10)



filtered out 19024 genes that are detected in less than 3 cells
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
regressing out ['total_counts', 'pct_counts_mt']
    sparse input is densified and may lead to high memory use


/Users/mariak/Documents/MDVmk/MDV/venv/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:641: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


    finished (0:00:02)


#### Principal component analysis

In [9]:
# running PCA
sc.tl.pca(adata, svd_solver="arpack")

computing PCA
    with n_comps=50
    finished (0:00:01)


#### Neighborhood graph

In [10]:
# computing the neighborhood graph of cells using the PCA representation of the anndata object
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

computing neighbors
    using 'X_pca' with n_pcs = 40
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)


#### Embedding the neighborhood graph in 2D

In [11]:
sc.tl.umap(adata)

computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:02)


#### Clustering the neighborhood graph

In [12]:
sc.tl.leiden(
    adata,
    resolution=0.9,
    random_state=0,
    #flavor="igraph", default is leidenalg
    n_iterations=2,
    directed=False,)

running Leiden clustering
    finished: found 7 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


/var/folders/xf/lj2gr_2s7txb29m_mkg78h_80000gn/T/ipykernel_83764/1228213169.py:1: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(


#### Ranking the highly differential genes in each cluster

In [13]:
sc.tl.rank_genes_groups(adata, "leiden", method="t-test")
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")
sc.tl.rank_genes_groups(adata, "leiden", method="logreg", max_iter=1000)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
 (0:00:01)


#### Converting the anndata object to pandas dataframes

In [14]:
# cells dataframe 

cells_df = pd.DataFrame(adata.obs)

# adding the PCA data to the cells dataframe
pca_np = np.array(adata.obsm["X_pca"])
cells_df["X_pca_1"] = pca_np[:, 0]
cells_df["X_pca_2"] = pca_np[:, 1]
cells_df["X_pca_3"] = pca_np[:, 2]

# adding the umap data to the cells dataframe
umap_np = np.array(adata.obsm["X_umap"])
cells_df["X_umap_1"] = umap_np[:, 0]
cells_df["X_umap_2"] = umap_np[:, 1]

cells_df["cell_id"] = adata.obs.index

assert isinstance(adata.X, np.ndarray)
adata.layers['counts'] = adata.X.copy()

## these can be added dynamically with the 'Add Gene Expr' UI in the frontend
assert isinstance(adata.layers['counts'], np.ndarray)
counts = np.array(adata.layers['counts'])
transpose_counts = np.transpose(counts)
cells_df['ARVCF']=transpose_counts[adata.var.index.get_loc("ARVCF")]
cells_df['DOK3']=transpose_counts[adata.var.index.get_loc("DOK3")]
cells_df['FAM210B']=transpose_counts[adata.var.index.get_loc("FAM210B")]
cells_df['GBGT1']=transpose_counts[adata.var.index.get_loc("GBGT1")]
cells_df['NFE2L2']=transpose_counts[adata.var.index.get_loc("NFE2L2")]
cells_df['UBE2D4']=transpose_counts[adata.var.index.get_loc("UBE2D4")]
cells_df['YPEL2']=transpose_counts[adata.var.index.get_loc("YPEL2")]

# genes dataframe
gene_table = adata.var
gene_table["gene_ids"]=gene_table.index

varm_np = np.array(adata.varm["PCs"])
gene_table["PCs_1"] = varm_np[:, 0]
gene_table["PCs_2"] = varm_np[:, 1]

## MDV charts set up section

The different types of charts are defined in the mdvtools/charts/ folder. These are:
1. Abundance box plot
2. Box plot
3. Density scatter plot
4. Dot plot
5. Heatmap plot
6. Histogram plot
7. Multi-line plot
8. Ring chart
9. Row chart
10. Sankey plot
11. Scatter plot
12. Scatter plot 3D
13. Stacked row plot
14. Table
15. Violin plot
16. Wordcloud

In [15]:
# Cells charts

# creating a row chart based on the Row Chart implementation to show the leiden clustering
row_chart = RowChart(
    title="leiden",
    param="leiden",
    position=[380, 270],
    size=[260, 260]
)

# configuring the row chart
row_chart.set_axis_properties("x", {"textSize": 13, "label": "", "tickfont": 10})


# creating a dot plot based on the DotPlot implementation to show the gene expression of selected gene markers
dot_plot = DotPlot(
    title="leiden",
    params=["leiden", "ARVCF", "DOK3", "FAM210B", "GBGT1", "NFE2L2", "UBE2D4", "YPEL2"],
    size=[400, 250],
    position=[10, 10]
)

# configuring the dot plot
dot_plot.set_axis_properties("x", {"label": "", "textSize": 13, "tickfont": 10})
dot_plot.set_axis_properties("y", {"label": "", "textSize": 13, "tickfont": 10})
dot_plot.set_axis_properties("ry", {"label": "", "textSize": 13, "tickfont": 10})
dot_plot.set_color_scale(log_scale=False)
dot_plot.set_color_legend(True, [40, 10])
dot_plot.set_fraction_legend(True, [0, 0])


# creating a histogram plot based on the HistogramPlot implementation to show the distribution of the number of genes per counts
histogram_plot = HistogramPlot(
    title="n_genes_by_counts",
    param="n_genes_by_counts",
    bin_number=17,
    display_min=0,
    display_max=2500,
    size=[360, 250],
    position=[10, 280]
)

# configuring the histogram
histogram_plot.set_x_axis(size=30, label="n_genes_by_counts", textsize=13, tickfont=10)
histogram_plot.set_y_axis(size=45, label="frequency", textsize=13, tickfont=10, rotate_labels=False)

# creating a scatter plot based on the ScatterPlot3D implementation to show the 3 PCA clustering components
scatter_plot = ScatterPlot3D(
    title="X_pca_1 x X_pca_2 x X_pca_3",
    params=["X_pca_1", "X_pca_2", "X_pca_3"],
    size=[250, 250],
    position=[420, 10],
    default_color="#377eb8",
    brush="default",
    center=[0, 0, 0],
    on_filter="hide",
    radius=5,
    opacity=0.8,
    axis_scales=[220, 220, 220],
    camera={"distance": 37543.999999999956, "theta": -1.038, "phi": 0.261}
)

# configuring the scatter plot
scatter_plot.set_color_by("leiden")


In [16]:
# Genes charts

# creating a scatter plot using the ScatterPlot implementation to show the 2 PCA clustering components
scatter2D_plot = ScatterPlot(
    title="PCs_1 x PCs_2",
    params=["PCs_1", "PCs_2"],
    size=[250, 250],
    position=[270, 10]
)

# configuring the scatter plot
scatter2D_plot.set_default_color("#377eb8")
scatter2D_plot.set_brush("poly")
scatter2D_plot.set_opacity(0.8)
scatter2D_plot.set_radius(2)
scatter2D_plot.set_color_legend(display=True, position=[20, 1])
scatter2D_plot.set_axis_properties("x", {"label": "PCs_1", "textSize": 13, "tickfont": 10})
scatter2D_plot.set_axis_properties("y", {"label": "PCs_2", "textSize": 13, "tickfont": 10})
scatter2D_plot.set_color_by("dispersions")

# creating a table using the TablePlot implementation to show all the data associated with the genes dataframe
table_plot = TablePlot(
    title="Data table",
    params= gene_table.columns.values.tolist() ,
    size=[250, 500],
    position=[10, 10]
)

# creating a histogram using the HistogramPlot implementation to show a histogram of the number of genes per cell distribution
histogram_plot_2 = HistogramPlot(
    title="n_cells",
    param="n_cells",
    bin_number=17,
    display_min=0,
    display_max=2500,
    size=[240, 240],
    position=[270, 280]
)

# configuring the histogram
histogram_plot_2.set_x_axis(size=30, label="n_cells", textsize=13, tickfont=10)
histogram_plot_2.set_y_axis(size=45, label="frequency", textsize=13, tickfont=10, rotate_labels=False)

## Set up and serve the MDV project

In [21]:
# setting up and serving the MDV project
base = os.path.expanduser('~/mdv')
project_path = os.path.join(base, 'pbmc3k') # defining the location where the project metadata will be stored
p = MDVProject(os.path.expanduser(project_path), delete_existing=True)

# adding the two data sources to the project
p.add_datasource("cells", cells_df)
p.add_datasource("genes", gene_table)

# converting the chart implementation outputs to JSON and setting up the project view
list_charts_cells = []
list_charts_genes = []

# cells panel
list_charts_cells.append(row_chart.plot_data)
list_charts_cells.append(dot_plot.plot_data)
list_charts_cells.append(histogram_plot.plot_data)
list_charts_cells.append(scatter_plot.plot_data)

# genes panel
list_charts_genes.append(table_plot.plot_data)
list_charts_genes.append(scatter2D_plot.plot_data)
list_charts_genes.append(histogram_plot_2.plot_data)

# setting the config combining the two panels
view_config = {'initialCharts': {"cells": list_charts_cells, "genes":list_charts_genes}}

# creating the link between the two datasets so that selecting a subset of genes to add the expression in cells is enabled
p.add_rows_as_columns_link("cells","genes","gene_ids","Gene Expression")
p.add_rows_as_columns_subgroup("cells","genes","Gene scores",adata.X) #add the gene expression 

# adding the view to the project configuration
p.set_view("default", view_config)
p.set_editable(True)

In [ ]:
# serving the project
# p.serve()


In [23]:
adata.X.shape

(2638, 1838)